In [135]:
import pandas as pd
import io
from pathlib import Path
import plotly.express as px

In [136]:
# Carga de Archivos y transformaciones generales
mes_comparable = 'Agosto'

#TENER CUIDADO A LA HORA DE SUBIR LA INFORMACION. EN ESTE CASO COMO VAMOS A REALIZAR UNA COMPARACION GENERAL POR TIENDA/FORMATO, NO ES NECESARIO APERTURAR EL REPORTE DE VENTAS Y VOLUMEN POR SECTOR SECCION. UNICAMENTE POR GF PARA QUITARLE LOS ENVASES AL VOLUMEN
df_ventas_y_volumen = pd.read_csv('data/ventas y volumen express 2023 - 2025.csv', encoding='utf-16', header=1)
df_debitos = pd.read_csv('data/debitos express 2023 - 2025.csv', encoding='utf-16', header=1, sep=',', decimal=',')
padron = pd.read_excel('data/padron - agosto.xlsx', header=17) #type:ignore

# Trabajo sobre Ventas y Volumen
#Me quedo unicamente con las columnas importantes
df_ventas_y_volumen = df_ventas_y_volumen[['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']]

#Renombro las columnas
df_ventas_y_volumen.columns = (df_ventas_y_volumen.columns.str.strip().str.lower().str.replace(" ", "_"))
df_ventas_y_volumen.rename(columns={
'ventas_c/impuesto':'venta',
'venta_en_unidades':'volumen'
}, inplace=True)

#Genero una columna para Obtener el ID tienda
df_ventas_y_volumen['numero_operacional'] = df_ventas_y_volumen['punto_operacional'].str.split('-').str[0]

#Me quedo con las columnas necesarias
ventas = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'venta']]
volumen = df_ventas_y_volumen[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'volumen']]

#Quito los NA de las columans de valores
ventas.dropna(subset=['venta'], how='any', inplace=True)
volumen.dropna(subset=['volumen'], how='any', inplace=True)

#Realizo transformaciones para quitar carateres y convertir las columnas a valores numericos
ventas['venta'] = ventas['venta'].str.replace('.','').str.replace(',','.').astype('float')
volumen['volumen'] = volumen['volumen'].str.split(',').str[0].str.replace('.','').astype('int')

#Renombro las columnas con valores de ambos DF
ventas.rename(columns={
'venta':'valores'
}, inplace=True)

volumen.rename(columns={
'volumen':'valores'
}, inplace=True)

#Categorizo los valores tanto de volumne como de Ventas
ventas['categoria'] = 'VCT'
volumen['categoria'] = 'VOL'

#Agrupo las ventas
ventas_agrupado = ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

#Quito Envases del Volumen y Agrupo
volumen_sin_vol = volumen[~volumen['grupo_de_familia'].str.contains('ENVASES')]
volumen_agrupado = volumen_sin_vol.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

# Trabajo sobre Debitos
# Renombro el DF
debitos_agrupados = df_debitos

# Renombro las columnas como corresponden
debitos_agrupados.columns = debitos_agrupados.columns.str.lower().str.replace(' ','_')

# Renombro la columna de Debitos a valores
debitos_agrupados.rename(columns={
'cant._tickets_por_local':'valores'
}, inplace=True)

# Genero una columna Categorica
debitos_agrupados['categoria'] = 'DEB'

# Genero columna para el ID tienda
debitos_agrupados['numero_operacional'] = debitos_agrupados['punto_operacional'].str.split('-').str[0]

# Ordeno las columnas del df
debitos_agrupados = debitos_agrupados[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

# Quito nulos numericos de la columna valores
debitos_agrupados.dropna(subset=['valores'], how='any', inplace=True)

# Convierto la columna de valores a su tipo de datos correspondiente
debitos_agrupados['valores'] = debitos_agrupados['valores'].str.replace('.','').astype(int)

# Trabajo sobre el padron
# Selecciono las columnas que me sirven del padron
padron = padron[['N°', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']] #type:ignore

# Cambio de nombres en el padron
padron.columns = (
padron.columns
.str.lower()
.str.strip()
.str.replace(' ', '_', regex=False)
.str.replace('m²', 'm', regex=False)
.str.replace('.2','')
)

# Formateo la fecha para que tenga sentido
padron['fecha_apertura'] = padron['fecha_apertura'].dt.strftime('%d/%m/%Y')

# Cambio el nombre de la columna N por "Numero Operacional"
padron.rename(columns={'n°':'numero_operacional'}, inplace=True)

# Quito los valores nulos utilizando como referencia la columna Numero Operacional, nombre y fecha apertura
padron.dropna(subset=['numero_operacional', 'nombre', 'fecha_apertura', mes_comparable[0:3].lower()], how='any', inplace=True)

# Genero una funcion para convertir los valores de una columna a mayuscula
def maysc(df: pd.DataFrame, columna: str):
    df[columna] = df[columna].str.upper()
#Aplico la formula a la columna del mes comparable para que todos los valores sean en mayuscula

maysc(padron, mes_comparable[0:3].lower())

# Coloco el numero operacional como numero
padron['numero_operacional'] = padron['numero_operacional'].astype(int)

# Concateno todos los df (venta, debito y volumen) y lo joineo con el padron
df = pd.concat([ventas_agrupado, volumen_agrupado, debitos_agrupados])

# Convierto el ID a numero
df['numero_operacional'] = df['numero_operacional'].astype(int)

# Genero el Join del df Agupado con el Padron con el objetivo de quedarme unicamente con aquellas tiendas Comparables
df_join = df.merge(padron, how='left', on='numero_operacional')

# Trabajo sobre Progresiones Total Formato
# Me quedo unicamente con las columnas que me sirven del DF Joineado (ACA TENGO LA SC DEL MES)
df_join = df_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'fecha_apertura', 'fin_de_cierre', 'provincia','categoria', 'valores', mes_comparable[0:3].lower()]]

# Filtro unicamente las lineas que sean Superficie Comparable
df_join_sc = df_join[df_join[mes_comparable[0:3].lower()] == 'SC']

#Renombro la Columna Mes a Fecha para Luego generar la Columna Mes Correspondiente
df_join_sc.rename(columns={
'mes':'fecha'
}, inplace=True)
df_join_sc['mes'] = df_join_sc['fecha'].str.split(' ').str[0]

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_12708\3922322100.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_12708\3922322100.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_12708\3922322100.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\juan_mera\AppData\L

### Formato

In [137]:
#Genero un diccionario con los meses y sus valores numericos de forma auxiliar
orden_meses = {"Enero":1, "Febrero":2, "Marzo":3, "Abril":4, "Mayo":5, "Junio":6, "Julio":7, "Agosto":8, "Septiembre":9, "Octubre":10, "Noviembre":11, "Diciembre":12}

#Genero una columna auziliar para ordenar los meses y luego limitar el periodo
df_join_sc['aux'] = df_join_sc['mes'].map(orden_meses)

# Agrupo por los campos que me sirven y pivoteo la info para calcular las progresiones
df_total_formato = df_join_sc.groupby(['año', 'mes', 'aux', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'aux', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero las progresiones
df_total_formato['progresion 2024'] = round((((df_total_formato[2024] / df_total_formato[2023]) - 1) * 100), 1)
df_total_formato['progresion 2025'] = round((((df_total_formato[2025] / df_total_formato[2024]) - 1) * 100), 1)

#Me traigo unicamente la informacion que me sirve
df_total_formato = df_total_formato[['mes', 'aux', 'categoria', 'progresion 2024', 'progresion 2025']]

#Ordeno el df
df_total_formato = df_total_formato.sort_values('aux')

#"Derrito" el df para poder generar un formato long y asi realizar un grafico con el periodo continuado desde 2024 en adelante
df_total_formato = df_total_formato.melt(id_vars=['mes', 'aux', 'categoria'], value_vars=['progresion 2024', 'progresion 2025'], var_name='progresiones', value_name='valores')

#genero una columna de periodo concatenando el mes y el año
df_total_formato['periodo'] = df_total_formato['aux'].astype(str).str.zfill(2) + '-' + df_total_formato['progresiones'].str.split(' ').str[1].astype(str).str[2:]

df_total_formato['punto_operacional'] = 'Total Formato'

df_total_formato = df_total_formato[['punto_operacional', 'mes', 'aux', 'categoria', 'progresiones', 'valores', 'periodo']]

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_12708\3021178585.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [138]:
# Agrupo por los campos que me sirven y pivoteo la info para calcular las progresiones
df_join_sc = df_join_sc.groupby(['año', 'mes', 'aux', 'punto_operacional', 'categoria'])['valores'].sum().reset_index().pivot_table(values='valores', index=['mes', 'aux', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()

#Genero las progresiones
df_join_sc['progresion 2024'] = round((((df_join_sc[2024] / df_join_sc[2023]) - 1) * 100), 1)
df_join_sc['progresion 2025'] = round((((df_join_sc[2025] / df_join_sc[2024]) - 1) * 100), 1)

#Me traigo unicamente la informacion que me sirve
df_progresiones = df_join_sc[['punto_operacional', 'mes', 'aux', 'categoria', 'progresion 2024', 'progresion 2025']]

#Ordeno el df
df_progresiones = df_progresiones.sort_values('aux')

#"Derrito" el df para poder generar un formato long y asi realizar un grafico con el periodo continuado desde 2024 en adelante
df_progresiones = df_progresiones.melt(id_vars=['punto_operacional', 'mes', 'aux', 'categoria'], value_vars=['progresion 2024', 'progresion 2025'], var_name='progresiones', value_name='valores')

#genero una columna de periodo concatenando el mes y el año
df_progresiones['periodo'] = df_progresiones['aux'].astype(str).str.zfill(2) + '-' + df_progresiones['progresiones'].str.split(' ').str[1].astype(str).str[2:]

In [139]:
df_final = pd.concat([df_progresiones, df_total_formato])

In [141]:
#Genero un df por tienda seleccionada (IMPORTANTE CAMBIAR ACA POR PARAMETROS EN LA FUNCION Y QUE SE ACTUALICE EN BASE A ESTO)
df_final_filtrado = df_final[(df_final['punto_operacional'].isin(['Total Formato', '400 - Proxi Centenera 177'])) & (df_final['categoria'] == 'VCT')]

In [143]:
fig = px.line(
    df_final_filtrado,
    x="periodo",
    y="valores",
    markers=True,
    text="valores",
    color='punto_operacional'
)

fig.update_layout(
    title={
        'text': f"Evolución {df_final_filtrado['categoria'].unique()[0]} vs Formato",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    yaxis_title="Valores en porcentaje",
    xaxis_title="Periodo (Mes-Año)"
)

fig.update_traces(textposition="top center")

fig